# Working with Open Model Zoo Models

In [3]:
# Install openvino package
%pip install -q "openvino-dev>=2023.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 40.7 MB/s eta 0:00:00


## Preparation [$\Uparrow$](#Table-of-content:)

### Model Name [$\Uparrow$](#Table-of-content:)

Set `model_name` to the name of the Open Model Zoo model to use in this notebook. Refer to the list of [public](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md) and [Intel](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md) pre-trained models for a full list of models that can be used. Set `model_name` to the model you want to use.

In [4]:
# model_name = "resnet-50-pytorch"
model_name = "mobilenet-v2-pytorch"

### Imports [$\Uparrow$](#Table-of-content:)


In [5]:
import json
from pathlib import Path

import openvino as ov
from IPython.display import Markdown, display

# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)
from notebook_utils import DeviceNotFoundAlert, NotebookAlert

### Settings and Configuration [$\Uparrow$](#Table-of-content:)

Set the file and directory paths. By default, this notebook downloads models from Open Model Zoo to the `open_model_zoo_models` directory in your `$HOME` directory. On Windows, the $HOME directory is usually `c:\users\username`, on Linux `/home/username`. To change the folder, change `base_model_dir` in the cell below.

The following settings can be changed:

* `base_model_dir`: Models will be downloaded into the `intel` and `public` folders in this directory.
* `omz_cache_dir`: Cache folder for Open Model Zoo. Specifying a cache directory is not required for Model Downloader and Model Converter, but it speeds up subsequent downloads.
* `precision`: If specified, only models with this precision will be downloaded and converted.

In [6]:
base_model_dir = Path("model")
omz_cache_dir = Path("cache")
precision = "FP16"

# Check if an iGPU is available on this system to use with Benchmark App.
core = ov.Core()
gpu_available = "GPU" in core.available_devices

print(
    f"base_model_dir: {base_model_dir}, omz_cache_dir: {omz_cache_dir}, gpu_availble: {gpu_available}"
)

base_model_dir: model, omz_cache_dir: cache, gpu_availble: False


## Download a Model from Open Model Zoo [$\Uparrow$](#Table-of-content:)



Specify, display and run the Model Downloader command to download the model.

In [ ]:
## Uncomment the next line to show help in omz_downloader which explains the command-line options.

# !omz_downloader --help

In [7]:
download_command = (
    f"omz_downloader --name {model_name} --output_dir {base_model_dir} --cache_dir {omz_cache_dir}"
)
display(Markdown(f"Download command: `{download_command}`"))
display(Markdown(f"Downloading {model_name}..."))
! $download_command

Download command: `omz_downloader --name mobilenet-v2-pytorch --output_dir model --cache_dir cache`

Downloading mobilenet-v2-pytorch...

################|| Downloading mobilenet-v2-pytorch ||################

========== Downloading model/public/mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth
... 100%, 13879 KB, 14866 KB/s, 0 seconds passed



## Convert a Model to OpenVINO IR format [$\Uparrow$](#Table-of-content:)

Specify, display and run the Model Converter command to convert the model to OpenVINO IR format. Model conversion may take a while. The output of the Model Converter command will be displayed. When the conversion is successful, the last lines of the output will include: `[ SUCCESS ] Generated IR version 11 model.` For downloaded models that are already in OpenVINO IR format, conversion will be skipped.

In [ ]:
## Uncomment the next line to show Help in omz_converter which explains the command-line options.

# !omz_converter --help

In [8]:
convert_command = f"omz_converter --name {model_name} --precisions {precision} --download_dir {base_model_dir} --output_dir {base_model_dir}"
display(Markdown(f"Convert command: `{convert_command}`"))
display(Markdown(f"Converting {model_name}..."))

! $convert_command

Convert command: `omz_converter --name mobilenet-v2-pytorch --precisions FP16 --download_dir model --output_dir model`

Converting mobilenet-v2-pytorch...

========== Converting mobilenet-v2-pytorch to ONNX
Conversion to ONNX command: /usr/bin/python3 -- /usr/local/lib/python3.10/dist-packages/openvino/model_zoo/internal_scripts/pytorch_to_onnx.py --model-name=mobilenet_v2 --weights=model/public/mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth --import-module=torchvision.models --input-shape=1,3,224,224 --output-file=model/public/mobilenet-v2-pytorch/mobilenet-v2.onnx --input-names=data --output-names=prob

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX check passed successfully.

========== Converting mobilenet-v2-pytorch to IR (FP16)
Conversion command: /usr/bin/python3 -- /usr/local/bin/mo --framework=onnx --output_dir=model/public/mobilenet-v2-pytorch/FP16 --model_name=mobilenet-v2-pytorch --input=data '--mean_values=data[123.675,116.28,103.53]' '--scale_values=data[58.624,57.12,

## Get Model Information [$\Uparrow$](#Table-of-content:)

The Info Dumper prints the following information for Open Model Zoo models:

* Model name
* Description
* Framework that was used to train the model
* License URL
* Precisions supported by the model
* Subdirectory: the location of the downloaded model
* Task type

This information can be shown by running `omz_info_dumper --name model_name` in a terminal. The information can also be parsed and used in scripts.

In the next cell, run Info Dumper and use `json` to load the information in a dictionary.

In [9]:
model_info_output = %sx omz_info_dumper --name $model_name
model_info = json.loads(model_info_output.get_nlstr())

if len(model_info) > 1:
    NotebookAlert(
        f"There are multiple IR files for the {model_name} model. The first model in the "
        "omz_info_dumper output will be used for benchmarking. Change "
        "`selected_model_info` in the cell below to select a different model from the list.",
        "warning",
    )

model_info

[{'name': 'mobilenet-v2-pytorch',
  'composite_model_name': None,
  'description': 'MobileNet V2 is image classification model pre-trained on ImageNet dataset. This is a PyTorch* implementation of MobileNetV2 architecture as described in the paper "Inverted Residuals and Linear Bottlenecks: Mobile Networks for Classification, Detection and Segmentation" <https://arxiv.org/abs/1801.04381>.\nThe model input is a blob that consists of a single image of "1, 3, 224, 224" in "RGB" order.\nThe model output is typical object classifier for the 1000 different classifications matching with those in the ImageNet database.',
  'framework': 'pytorch',
  'license_url': 'https://raw.githubusercontent.com/pytorch/vision/master/LICENSE',
  'accuracy_config': '/usr/local/lib/python3.10/dist-packages/openvino/model_zoo/models/public/mobilenet-v2-pytorch/accuracy-check.yml',
  'model_config': '/usr/local/lib/python3.10/dist-packages/openvino/model_zoo/models/public/mobilenet-v2-pytorch/model.yml',
  'prec

Having information of the model in a JSON file enables extraction of the path to the model directory, and building the path to the OpenVINO IR file.

In [10]:
selected_model_info = model_info[0]
model_path = (
    base_model_dir
    / Path(selected_model_info["subdirectory"])
    / Path(f"{precision}/{selected_model_info['name']}.xml")
)
print(model_path, "exists:", model_path.exists())

model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml exists: True


## Run Benchmark Tool [$\Uparrow$](#Table-of-content:)

By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput values (frames per second).

In [ ]:
## Uncomment the next line to show Help in benchmark_app which explains the command-line options.
# !benchmark_app --help

In [18]:
benchmark_command = f"benchmark_app -m {model_path} -t 60"
display(Markdown(f"Benchmark command: `{benchmark_command}`"))
display(Markdown(f"Benchmarking {model_name} on CPU with async inference for 60 seconds..."))

! $benchmark_command

Benchmark command: `benchmark_app -m model/public/mobilenet-v2-pytorch/FP16/mobilenet-v2-pytorch.xml -t 60`

Benchmarking mobilenet-v2-pytorch on CPU with async inference for 60 seconds...

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2023.1.0-12185-9e6b00e51cd-releases/2023/1
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2023.1.0-12185-9e6b00e51cd-releases/2023/1
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 20.10 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     data (node: data) : f32 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     prob (node: prob) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring in